In [ ]:
import torch as ch

In [ ]:
resume_path = 'model_best.pth'

ckpt = ch.load(resume_path, map_location='cpu')
# start_epoch = ckpt['epoch']
# model.load_state_dict(update_state_dict(model.state_dict(),ckpt['model']))
# for group_name in prob_models:
#                     prob_models[group_name].load_state_dict(update_state_dict(prob_models[group_name].state_dict(),ckpt['prob_models'][group_name]))
# optimizer.load_state_dict(ckpt['optimizer'])
# prob_optimizer.load_state_dict(ckpt['prob_optimizer'])

In [ ]:
ckpt.keys()

dict_keys(['epoch', 'model', 'prob_models', 'best_acc1', 'best_epoch', 'optimizer', 'prob_optimizer', 'bits', 'ac_bytes'])

In [ ]:
ckpt['bits']

tensor(95003.7344)

In [ ]:
ckpt['bits']/8000

tensor(11.8755)

In [ ]:
ckpt['ac_bytes']

11734

In [ ]:
model = ckpt['model']

In [ ]:
model.keys()

odict_keys(['module.feature_extraction_layer.0.weight', 'module.feature_extraction_layer.0.bias', 'module.feature_extraction_layer.0.weight_decoder.scale', 'module.feature_extraction_layer.2.weight', 'module.feature_extraction_layer.2.bias', 'module.feature_extraction_layer.2.running_mean', 'module.feature_extraction_layer.2.running_var', 'module.feature_extraction_layer.2.num_batches_tracked', 'module.feature_extraction_layer.4.weight', 'module.feature_extraction_layer.4.bias', 'module.feature_extraction_layer.4.weight_decoder.scale', 'module.feature_extraction_layer.6.weight', 'module.feature_extraction_layer.6.bias', 'module.feature_extraction_layer.6.running_mean', 'module.feature_extraction_layer.6.running_var', 'module.feature_extraction_layer.6.num_batches_tracked', 'module.binary_classification_layer.1.weight', 'module.binary_classification_layer.1.bias', 'module.binary_classification_layer.1.weight_decoder.scale', 'module.binary_classification_layer.4.weight', 'module.binary_c

In [ ]:
layer_0_weights = 'module.feature_extraction_layer.0.weight'
layer_2_weights = 'module.feature_extraction_layer.2.weight'
layer_4_weights = 'module.feature_extraction_layer.4.weight'
layer_4_weigths_decoder = 'module.feature_extraction_layer.4.weight_decoder.scale'

classification_layer_1_weight = 'module.binary_classification_layer.1.weight'

In [ ]:
model[layer_4_weights].shape

torch.Size([64, 32, 5, 5])

In [ ]:
conv_weights = ['module.feature_extraction_layer.0.weight', 'module.feature_extraction_layer.4.weight']

for curr_conv_weights in conv_weights:

  param = ch.round(model[curr_conv_weights])
  weight_oi = ch.sum(ch.abs(param),dim=(2,3))
  non_zero_in = ch.count_nonzero(ch.sum(weight_oi, dim=0))
  non_zero_out = ch.count_nonzero(ch.sum(weight_oi, dim=1))

  print(f"Conv2d(in_channels={non_zero_in}, out_channels={non_zero_out})")

in_features = 11*29*non_zero_out
print(f"Linear(in_features={in_features}, out_features=64)")

Conv2d(in_channels=1, out_channels=27)
Conv2d(in_channels=27, out_channels=26)
Linear(in_features=8294, out_features=64)


# Model size

In [ ]:
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class ConvNetIDS(nn.Module):
    def __init__(self, width=1, init_type='random', compress_bias = False, vanilla=False,\
                 mode='fan_out', boundary=10, no_shift=False):
        super(ConvNetIDS, self).__init__()

        self.feature_extraction_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32, eps=0.001, momentum=0.9),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64, eps=0.001, momentum=0.9),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.binary_classification_layer = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=20416, out_features=64),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )

    def forward(self, x):
        x = self.feature_extraction_layer(x)
        x = torch.flatten(x, 1)
        x = self.binary_classification_layer(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
model = ConvNetIDS()

In [ ]:
from pytorch_modelsize import SizeEstimator

In [ ]:
estimator = SizeEstimator(model, input_size=(1, 1, 44, 116))

In [ ]:
estimator.calc_param_bits()

In [ ]:
total = estimator.param_bits

In [ ]:
print((total/8)/(1024))

10617.5078125
